In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [ ]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:,-1].values

In [ ]:
x.shape

(10000, 10)

In [ ]:
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
x[:,2] = le.fit_transform(x[:,2])

In [ ]:
x

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [ ]:
x

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape

(8000, 12)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train[0]

array([-1.01460667, -0.5698444 ,  1.74309049,  0.16958176, -1.09168714,
       -0.46460796,  0.00666099, -1.21571749,  0.8095029 ,  0.64259497,
       -1.03227043,  1.10643166])

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))  

#Since we are dealing with a BINARY CLASSIFICATION problem, so 1 neuron can work.
#In case of MULTI-CLASSIFICATION problem; units=no. of classes
#For binary, sigmoid is used.

In [ ]:
ann.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy']) #loss='categorical_crossentropy' in case of multi-class problem
ann.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 5s 2ms/step - loss: 0.6644 - accuracy: 0.5974
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4729 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4439 - accuracy: 0.7962
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4324 - accuracy: 0.8051
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4278 - accuracy: 0.8125
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4254 - accuracy: 0.8146
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4234 - accuracy: 0.8173
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4219 - accuracy: 0.8198
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4205 - accuracy: 0.8201
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4198 - accura

In [ ]:
print(ann.predict(sc.transform([[1,0,0, 600, 1, 40, 3, 60000,2,1,1,50000]]))>0.5)  # .predict always deal in 2D matrix

[[False]]


In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), Y_test.reshape(len(Y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cn = confusion_matrix(Y_test, y_pred)
print(cn)
acc = accuracy_score(Y_test, y_pred)
print(acc)

[[1506   89]
 [ 188  217]]
0.8615
